In [195]:
# custom modules
from ranking_score import *
from iaa import *

In [29]:
import pandas as pd
pd.set_option("display.max_rows", None)

import numpy as np
from koMecab import mecab
import sacrebleu

### Get the data

In [21]:
before = pd.read_csv("./raw_data/relative_ranking_a+b_before.csv", sep="\t", encoding="utf-8")
after = pd.read_csv("./raw_data/relative_ranking_a+b_after.csv", sep="\t", encoding="utf-8")

In [22]:
del before["Unnamed: 0"]
del after["Unnamed: 0"]
before.head(1)

,Segment ID,Source Segment,Ref,MT_Y,MT_Z,Ref_rating,MT_Y_rating,MT_Z_rating,Error
0,1,"Royal news: How Princess Beatrice ""reacted wit...","영국 왕실 뉴스: 베아트리스 공주, ‘적대적인 왕실 구성원들에게 이렇게 분노 표출해’","왕실 뉴스: 베아트리체 공주가 ""적대하는 가족들에게 분노의 반응을 보였다""",왕실 뉴스: 베아트리체 공주가 적대적인 가족들에게 분노로 어떻게 반응했는가,2,2,1,F


### Get the result

In [162]:
result_before = get_result(before)
result_after = get_result(after)

In [178]:
result_before

,MT_Y,MT_Z
MT_better,61,56
HT_better,256,284
Tie,120,97
P-value,3.62841e-21,5.10358e-29
Valid,True,True
Ab_score,2.22883,2.07551


In [179]:
result_after

,MT_Y,MT_Z
MT_better,49,40
HT_better,290,321
Tie,98,76
P-value,3.71747e-32,3.81629e-44
Valid,True,True
Ab_score,2.13959,1.93593


#### absolute score of HT

In [181]:
absolute_score(before, "Ref")

{'Ref': (2.7734553775743707, 360, 55, 22)}

In [182]:
absolute_score(after, "Ref")

{'Ref': (2.8260869565217392, 380, 38, 19)}

### Result: Error=="T"

In [170]:
before_error = before[before["Error"] == "T"]
len(before_error)

92

In [171]:
after_error = after[after["Error"] == "T"]
len(after_error)

92

In [183]:
result_before_error = get_result(before_error)
result_after_error = get_result(after_error)

In [184]:
result_before_error

,MT_Y,MT_Z
MT_better,18,12
HT_better,58,59
Tie,16,21
P-value,3.67005e-05,5.34814e-07
Valid,True,True
Ab_score,2.18478,2.09783


In [185]:
result_after_error

,MT_Y,MT_Z
MT_better,9,6
HT_better,61,68
Tie,22,18
P-value,4.60625e-08,3.2624e-11
Valid,True,True
Ab_score,2.11957,1.94565


#### absolute score of HT

In [187]:
absolute_score(before_error, "Ref")

{'Ref': (2.7065217391304346, 74, 9, 9)}

In [186]:
absolute_score(after_error, "Ref")

{'Ref': (2.858695652173913, 82, 7, 3)}

### IAA

In [10]:
file_dicts = read_files()  # group, dfs
individual_IAA(file_dicts, "a")

0.4759

In [199]:
file_dicts.keys()

dict_keys(['a1', 'a2', 'b1', 'b2', 'c1', 'c2', 'd1', 'd2', 'e1', 'e2', 'f1', 'f2', 'g1', 'g2', 'h1', 'h2', 'i1', 'i2', 'j1', 'j2', 'k1', 'k2'])

In [11]:
final_IAA(file_dicts)

{'a': 0.4759,
 'b': 0.3032,
 'c': 0.4107,
 'd': 0.1687,
 'e': 0.2711,
 'f': 0.382,
 'g': 0.3773,
 'h': 0.1314,
 'i': 0.2233,
 'j': 0.1971,
 'k': 0.474}

In [16]:
kappa_dict = final_IAA(file_dicts)
print(np.mean([score for _, score in kappa_dict.items()]))
print(np.min([score for _, score in kappa_dict.items()]))
print(np.max([score for _, score in kappa_dict.items()]))

0.3104272727272727
0.1314
0.4759


### SacreBLEU

#### before

In [121]:
%%time
reference = before.Ref.tolist()
hypothesis_y = before.MT_Y.tolist()
hypothesis_z = before.MT_Z.tolist()

reference = [' '.join(mecab(s)).replace("▃", "").replace("  ", " ") for s in reference]
hypothesis_y = [' '.join(mecab(s)).replace("▃", "").replace("  ", " ") for s in hypothesis_y]
hypothesis_z = [' '.join(mecab(s)).replace("▃", "").replace("  ", " ") for s in hypothesis_z]

for system in [hypothesis_y, hypothesis_z]:
    
    bleu = sacrebleu.corpus_bleu(system, [reference])
    ter = sacrebleu.corpus_ter(system, [reference])
    !--force
    chrf = sacrebleu.corpus_chrf(system, [reference])
    
    if system == hypothesis_y:
        print("[MT_Y]")
    else:
        print("[MT_Z]")
        
    print("BLEU: ", bleu.score)
    print("TER: ", ter.score)
    print("chrF", chrf.score)
    print()

That's 100 lines that end in a tokenized period ('.')
It looks like you forgot to detokenize your test data, which may hurt your score.
If you insist your data is detokenized, or don't care, you can suppress this message with '--force'.


/bin/bash: --: invalid option
Usage:	/bin/bash [GNU long option] [option] ...
	/bin/bash [GNU long option] [option] script-file ...
GNU long options:
	--debug
	--debugger
	--dump-po-strings
	--dump-strings
	--help
	--init-file
	--login
	--noediting
	--noprofile
	--norc
	--posix
	--rcfile
	--restricted
	--verbose
	--version
Shell options:
	-ilrsD or -c command or -O shopt_option		(invocation only)
	-abefhkmnptuvxBCHP or -o option


That's 100 lines that end in a tokenized period ('.')
It looks like you forgot to detokenize your test data, which may hurt your score.
If you insist your data is detokenized, or don't care, you can suppress this message with '--force'.


[MT_Y]
BLEU:  19.785266218415302
TER:  0.6547766323024055
chrF 0.2707334386314116

/bin/bash: --: invalid option
Usage:	/bin/bash [GNU long option] [option] ...
	/bin/bash [GNU long option] [option] script-file ...
GNU long options:
	--debug
	--debugger
	--dump-po-strings
	--dump-strings
	--help
	--init-file
	--login
	--noediting
	--noprofile
	--norc
	--posix
	--rcfile
	--restricted
	--verbose
	--version
Shell options:
	-ilrsD or -c command or -O shopt_option		(invocation only)
	-abefhkmnptuvxBCHP or -o option
[MT_Z]
BLEU:  20.19309663896737
TER:  0.6626804123711341
chrF 0.26968267974474885

CPU times: user 43.2 s, sys: 290 ms, total: 43.5 s
Wall time: 43.7 s


#### after

In [120]:
%%time
reference = after.Ref.tolist()
hypothesis_y = after.MT_Y.tolist()
hypothesis_z = after.MT_Z.tolist()

reference = [' '.join(mecab(s)).replace("▃", "").replace("  ", " ") for s in reference]
hypothesis_y = [' '.join(mecab(s)).replace("▃", "").replace("  ", " ") for s in hypothesis_y]
hypothesis_z = [' '.join(mecab(s)).replace("▃", "").replace("  ", " ") for s in hypothesis_z]

for system in [hypothesis_y, hypothesis_z]:
    
    bleu = sacrebleu.corpus_bleu(system, [reference])
    ter = sacrebleu.corpus_ter(system, [reference])
    !--force
    chrf = sacrebleu.corpus_chrf(system, [reference])
    
    if system == hypothesis_y:
        print("[MT_Y]")
    else:
        print("[MT_Z]")
        
    print("BLEU: ", bleu.score)
    print("TER: ", ter.score)
    print("chrF", chrf.score)
    print()

That's 100 lines that end in a tokenized period ('.')
It looks like you forgot to detokenize your test data, which may hurt your score.
If you insist your data is detokenized, or don't care, you can suppress this message with '--force'.


/bin/bash: --: invalid option
Usage:	/bin/bash [GNU long option] [option] ...
	/bin/bash [GNU long option] [option] script-file ...
GNU long options:
	--debug
	--debugger
	--dump-po-strings
	--dump-strings
	--help
	--init-file
	--login
	--noediting
	--noprofile
	--norc
	--posix
	--rcfile
	--restricted
	--verbose
	--version
Shell options:
	-ilrsD or -c command or -O shopt_option		(invocation only)
	-abefhkmnptuvxBCHP or -o option


That's 100 lines that end in a tokenized period ('.')
It looks like you forgot to detokenize your test data, which may hurt your score.
If you insist your data is detokenized, or don't care, you can suppress this message with '--force'.


[MT_Y]
BLEU:  19.77921498933438
TER:  0.6520962199312714
chrF 0.271264889552056

/bin/bash: --: invalid option
Usage:	/bin/bash [GNU long option] [option] ...
	/bin/bash [GNU long option] [option] script-file ...
GNU long options:
	--debug
	--debugger
	--dump-po-strings
	--dump-strings
	--help
	--init-file
	--login
	--noediting
	--noprofile
	--norc
	--posix
	--rcfile
	--restricted
	--verbose
	--version
Shell options:
	-ilrsD or -c command or -O shopt_option		(invocation only)
	-abefhkmnptuvxBCHP or -o option
[MT_Z]
BLEU:  20.241028636714525
TER:  0.6623367697594502
chrF 0.2709943645290349

CPU times: user 42.3 s, sys: 334 ms, total: 42.6 s
Wall time: 42.8 s


#### Google Translate

In [175]:
df = pd.read_csv("./data_out/_google+ter.csv", sep="\t", encoding="utf-8")
del df["Unnamed: 0"]
df.head(1)

,Google Before,Google After,TER
0,"왕실 뉴스: 베아트리체 공주가 어떻게 ""적대적인 가족 구성원들에게 분노에 반응했는지""",Royal News : 공주님이 적대 가족 구성원들에게 분노와 반응 한 공주,30.84375


In [167]:
len(df)

437

In [168]:
reference = after.Ref.tolist()
google_before = df["Google Before"].tolist()
google_after = df["Google After"].tolist()

reference = [' '.join(mecab(s)).replace("▃", "").replace("  ", " ") for s in reference]
google_before = [' '.join(mecab(s)).replace("▃", "").replace("  ", " ") for s in google_before]
google_after = [' '.join(mecab(s)).replace("▃", "").replace("  ", " ") for s in google_after]

for case in [google_before, google_after]:
    
    bleu = sacrebleu.corpus_bleu(case, [reference])
    ter = sacrebleu.corpus_ter(case, [reference])
    !--force
    chrf = sacrebleu.corpus_chrf(case, [reference])
    
    if case == google_before:
        print("[Google Before]")
    else:
        print("[Google After]")
        
    print("BLEU: ", bleu.score)
    print("TER: ", ter.score)
    print("chrF", chrf.score)
    print()

That's 100 lines that end in a tokenized period ('.')
It looks like you forgot to detokenize your test data, which may hurt your score.
If you insist your data is detokenized, or don't care, you can suppress this message with '--force'.


/bin/bash: --: invalid option
Usage:	/bin/bash [GNU long option] [option] ...
	/bin/bash [GNU long option] [option] script-file ...
GNU long options:
	--debug
	--debugger
	--dump-po-strings
	--dump-strings
	--help
	--init-file
	--login
	--noediting
	--noprofile
	--norc
	--posix
	--rcfile
	--restricted
	--verbose
	--version
Shell options:
	-ilrsD or -c command or -O shopt_option		(invocation only)
	-abefhkmnptuvxBCHP or -o option


That's 100 lines that end in a tokenized period ('.')
It looks like you forgot to detokenize your test data, which may hurt your score.
If you insist your data is detokenized, or don't care, you can suppress this message with '--force'.


[Google Before]
BLEU:  13.117676138341224
TER:  0.7595189003436427
chrF 0.19496856536087329

/bin/bash: --: invalid option
Usage:	/bin/bash [GNU long option] [option] ...
	/bin/bash [GNU long option] [option] script-file ...
GNU long options:
	--debug
	--debugger
	--dump-po-strings
	--dump-strings
	--help
	--init-file
	--login
	--noediting
	--noprofile
	--norc
	--posix
	--rcfile
	--restricted
	--verbose
	--version
Shell options:
	-ilrsD or -c command or -O shopt_option		(invocation only)
	-abefhkmnptuvxBCHP or -o option
[Google After]
BLEU:  9.492515518561063
TER:  0.8156701030927835
chrF 0.1588497884299075



### Qualitative Analysis with TER

#### google

In [169]:
ter=[]
for bts, ats in zip(google_before, google_after):
    score = sacrebleu.sentence_ter(bts, ats)
    ter.append(score.score)
    
df["TER"] = ter
df.head(1)

,Google Before,Google After,TER
0,"왕실 뉴스: 베아트리체 공주가 어떻게 ""적대적인 가족 구성원들에게 분노에 반응했는지""",Royal News : 공주님이 적대 가족 구성원들에게 분노와 반응 한 공주,30.84375


In [170]:
df[df.TER >= 100]

,Google Before,Google After,TER
42,HSBC의 클라이언트 네트워크 뱅킹 글로벌 책임자인 스튜어트 니비슨(Stuart N...,"""그것은 금융 범죄 예방에 대한 투자의 첫 번째 상업적 용도 중 하나이며, 우리 가...",106.144330
128,"""이 상은 정치적 설득이 전혀 다른 공적 삶의 사람들이 자신의 주장을 품위와 정중함...","""이러한상은 사례와 정중함으로 사건을 주장하는 정치인들에 대한 스포트라이트를 비추고...",106.600000
316,프레드 마이어(Fred Meyer) 슈퍼마켓 체인의 식료품점 노동자들은 고용주에 대...,Fred Meyer 슈퍼마켓 체인의 식료품 점 노동자는 고용주에 대한 캠페인을 늘리...,109.935484
333,그녀는 Meghan이 참석하는 것이 '계층적 느낌이 들지 않았다'고 덧붙였습니다. ...,"그녀는 ""계층 적으로 느끼지 못한다""라고 덧붙였다. 그것은 ""우리가 가진 여성들과 ...",105.447619
364,"대변인은 다음과 같이 말했습니다. ""이제 가능합니다 - 정치적 의지가있을 경우 - ...",대변인은 다음과 같이 말했습니다. Benn Act 주위를 얻는 방법은 일시적인 Ca...,146.571429
382,전반적으로 폐하의 좌우명 및 화재 및 구조 서비스 (HMICFR)의 폐하의 검사관이...,전반적으로 폐하의 좌우명 및 화재 및 구조 서비스 (HMICFR)의 폐하의 검사관이...,101.163462
400,"그는 ""공정성과 존경심으로 인력을 지속적으로 취급하지 않으며 효과적으로 인력과 의사...","그는 ""공정성과 존경심으로 인력을 지속적으로 취급하지 않으며 효과적으로 인력과 의사...",102.463158


In [172]:
#df.to_csv("./data_out/_google+ter.csv", sep="\t", encoding="utf-8")

#### MT_Z & MT_Y

In [133]:
before.head(1)

,Segment ID,Source Segment,Ref,MT_Y,MT_Z,Ref_rating,MT_Y_rating,MT_Z_rating,Error
0,1,"Royal news: How Princess Beatrice ""reacted wit...","영국 왕실 뉴스: 베아트리스 공주, ‘적대적인 왕실 구성원들에게 이렇게 분노 표출해’","왕실 뉴스: 베아트리체 공주가 ""적대하는 가족들에게 분노의 반응을 보였다""",왕실 뉴스: 베아트리체 공주가 적대적인 가족들에게 분노로 어떻게 반응했는가,2,2,1,F


In [196]:
bts_y = before.MT_Y.tolist()
ats_y = after.MT_Y.tolist()

bts_y = [' '.join(mecab(s)).replace("▃", "").replace("  ", " ") for s in bts_y]
ats_y = [' '.join(mecab(s)).replace("▃", "").replace("  ", " ") for s in ats_y]


ter=[]
for bts, ats in zip(bts_y, ats_y):
    score = sacrebleu.sentence_ter(bts, ats).score
    ter.append(score)
    
before["MT_Y_TER"] = ter
before.head(1)

,Segment ID,Source Segment,Ref,MT_Y,MT_Z,Ref_rating,MT_Y_rating,MT_Z_rating,Error,MT_Y_TER,MT_Z_TER
0,1,"Royal news: How Princess Beatrice ""reacted with fury at hostile family members""","영국 왕실 뉴스: 베아트리스 공주, ‘적대적인 왕실 구성원들에게 이렇게 분노 표출해’","왕실 뉴스: 베아트리체 공주가 ""적대하는 가족들에게 분노의 반응을 보였다""",왕실 뉴스: 베아트리체 공주가 적대적인 가족들에게 분노로 어떻게 반응했는가,2,2,1,F,29.939394,25.757576


In [150]:
bts_z = before.MT_Z.tolist()
ats_z = after.MT_Z.tolist()

bts_z = [' '.join(mecab(s)).replace("▃", "").replace("  ", " ") for s in bts_z]
ats_z = [' '.join(mecab(s)).replace("▃", "").replace("  ", " ") for s in ats_z]

ter=[]
for bts, ats in zip(bts_z, ats_z):
    score = sacrebleu.sentence_ter(bts, ats)
    ter.append(score.score)
    
before["MT_Z_TER"] = ter
before.head(1)

,Segment ID,Source Segment,Ref,MT_Y,MT_Z,Ref_rating,MT_Y_rating,MT_Z_rating,Error,MT_Y_TER,MT_Z_TER
0,1,"Royal news: How Princess Beatrice ""reacted wit...","영국 왕실 뉴스: 베아트리스 공주, ‘적대적인 왕실 구성원들에게 이렇게 분노 표출해’","왕실 뉴스: 베아트리체 공주가 ""적대하는 가족들에게 분노의 반응을 보였다""",왕실 뉴스: 베아트리체 공주가 적대적인 가족들에게 분노로 어떻게 반응했는가,2,2,1,F,29.939394,25.757576


In [153]:
df = before
print("Y= ", np.max(df["MT_Y_TER"].tolist()))
print("Z= ", np.max(df["MT_Z_TER"].tolist()))

Y=  123.15789473684211
Z=  161.5


In [159]:
len(df[(df["MT_Y_TER"] >= 100) | (df["MT_Z_TER"] >= 100)])

13

In [160]:
df[(df["MT_Y_TER"] >= 100) | (df["MT_Z_TER"] >= 100)]

,Segment ID,Source Segment,Ref,MT_Y,MT_Z,Ref_rating,MT_Y_rating,MT_Z_rating,Error,MT_Y_TER,MT_Z_TER
42,43,It's one of the first commercial uses of inves...,HSBC 고객네트워크뱅킹 부서의 스튜어트 나이비슨 글로벌사업 그룹장은 “이번이 금융...,"HSBC의 글로벌 고객 네트워크 뱅킹 책임자인 스튜어트 니비슨은 ""금융 범죄 예방에...","스튜어트 니비슨 HSBC 글로벌 고객 네트워크 뱅킹 대표는 ""금융범죄 예방에 대한 ...",1,2,3,F,103.009524,91.235955
128,129,These awards are a small attempt by a group of...,"“이 상은 자신의 주장을 펼침에 있어 품위와 예의를 갖추는 정치인, 당파를 넘어 모...","""이러한 상들은 공직생활에 종사하는 사람들의 작은 시도입니다. 정치적 설득이 전혀 ...","""이 상들은 다른 정치적 설득력이 없는 공공 생활에서 한 무리의 사람들이 품위와 예...",1,2,3,F,123.157895,111.622642
258,259,The Centres for Disease Control and Prevention...,미국 질병통제예방센터에서는 현재 100여 명의 담당자들이 정체불명의 이 질환의 원인...,질병통제예방센터(CDC)는 100명 이상의 관리들이 이 미스터리 질병의 원인을 조사...,질병통제예방센터(CDC)는 미스터리 질병의 원인을 조사하는 100명 이상의 관리를 ...,1,2,3,F,103.384615,99.360825
279,280,"Citing a U.N.-backed reportreleased last year,...","지난해 공개된 UN 후원 보고서에 따르면, 시위자들은 지구 온도가 2.7도 이상 상...","시위대는 지난해 유엔이 지지한 보고서를 인용, 지구온도가 2.7도 이상 상승하는 것...","시위대는 지난해 발표된 유엔의 지원을 받는 보고서를 인용, 지구 기온이 2.7도 이...",1,1,2,T,102.808989,112.000000
293,294,While we are trying to establish a motive for ...,"“사건의 범행 동기를 찾으려고 노력 중이지만, 우리는 이번 사건을 표적 범죄로 보고...","그는 ""우리는 이번 사건의 동기를 밝히기 위해 노력하고 있지만 이번 사건이 목표가 ...","""우리가 일어난 일에 대한 동기를 확립하려고 노력하는 동안, 우리는 그 사건이 표적...",1,2,3,T,92.162162,103.571429
316,317,Grocery store workers at the Fred Meyer superm...,"대형마트 체인인 프레드 메이어의 직원들도 고용주를 상대로 캠페인을 벌이고 있으며, ...",프레드 마이어 슈퍼마켓 체인의 식료품점 노동자들은 고용주에 대한 반대 운동을 강화하...,프레드 마이어 슈퍼마켓 체인의 식료품점 근로자들은 고용주에 대한 캠페인을 강화하고 ...,2,1,2,F,102.389381,116.325203
317,318,"In the healthcare sector, the 4,500 members of...","의료계에서는 카이저 퍼머넌트의 노조인 서비스종업원 국제연맹49 지부의 회원 4,50...","의료 분야에서는, 카이저 퍼머넌트의 4,500명의 국제노동조합(ISU) 회원들이 1...","의료 분야에서는 카이저 퍼머넌트의 4,500명의 서비스 직원 국제 연합 지역 49명...",1,2,2,F,89.720430,109.349057
331,332,"Mother-of-two Naadiya Moosajee, a South Africa...",남아공 출신 공학자이자 기업가이며 위민 인 엔지니어링의 공동설립자인 두 아이의 엄마...,남아프리카의 엔지니어이자 사회적 기업가이자 여성 공학 분야의 공동 설립자인 Naad...,"남아프리카의 엔지니어, 사회적 기업가, 공학 분야의 공동 설립자인 두 아이의 어머니...",1,2,2,T,115.676923,120.225564
333,334,She added that having Meghan present 'didn't f...,"또한 그녀는 메건의 존재가 “위계적으로 느끼지 않았다”, 왜냐하면 여기는 “여성들이...",그녀는 메건이 참석하는 것이 '여성들이 함께 모여서 우리가 가진 투쟁과 앞으로 나아...,그녀는 메건이 '여성들이 함께 모여 우리가 가진 투쟁과 앞으로 나아가고 성장하고 사...,2,1,3,T,106.179775,93.333333
335,336,'She talked about being a mother and having du...,"“메건은 엄마가 되는 것, 공작 부인으로서의 의무, 자신의 가슴이 원하는 일을 하는...",'그녀는 엄마가 되어 공작부인으로서의 의무를 갖고 그녀의 마음이 원하는 것을 성취하...,'그녀는 엄마가 되고 공작부인으로서의 의무를 가지고 그녀의 마음이 원하는 것을 성취...,1,2,1,T,93.231707,101.162791


In [171]:
#df.to_csv("./data_out/_before+ter.csv", sep="\t", encoding="utf-8")

In [181]:
pd.set_option("display.max_colwidth", None)
before[before["Segment ID"] == 332]

,Segment ID,Source Segment,Ref,MT_Y,MT_Z,Ref_rating,MT_Y_rating,MT_Z_rating,Error,MT_Y_TER,MT_Z_TER
331,332,"Mother-of-two Naadiya Moosajee, a South African engineer, social entrepreneur and co-founder of Women in Engineering, revealed: 'We acknowledged the importance of not just education for girls but being role models for boys - that we are women who are influential and who boys can look up to as well,' People reports.","남아공 출신 공학자이자 기업가이며 위민 인 엔지니어링의 공동설립자인 두 아이의 엄마 나디야 무사지는 다음과 같이 말했다. “우리는 소녀들을 교육시키는 것뿐만 아니라 소년들의 롤모델이 되는 것도 중요하다는 걸 안다. 왜냐하면 우리는 영향력 있는 여성들이고, 소년들이 존경할 수 있는 사람들이기 때문이다.”라고 피플지(People)는 보도했다.",남아프리카의 엔지니어이자 사회적 기업가이자 여성 공학 분야의 공동 설립자인 Naadiya Mosajee는 '우리는 여자아이들을 위한 교육뿐만 아니라 남자아이들을 위한 역할 모델이 되는 것의 중요성을 인정했습니다 - 우리는 영향력 있고 남자아이들도 존경할 수 있는 여자입니다'라고 밝혔다.,"남아프리카의 엔지니어, 사회적 기업가, 공학 분야의 공동 설립자인 두 아이의 어머니 나디야 무사지는 '우리는 소녀들을 위한 교육뿐만 아니라 소년들을 위한 역할 모델이 되는 것의 중요성을 인정했다. 우리는 영향력 있고 소년들이 존경할 수 있는 여성이다'라고 사람들은 보고한다.",1,2,2,T,115.676923,120.225564


In [182]:
after[after["Segment ID"] == 332]

,Segment ID,Source Segment,Ref,MT_Y,MT_Z,Ref_rating,MT_Y_rating,MT_Z_rating,Error,MT_Y_TER,MT_Z_TER
331,332,"Mother-of-two Naadiya Moosajee, a South African engineer, social entrepreneur and co-founder of Women in Engineering, revealed: “We acknowledged the importance of not just education for girls but being role models for boys — that we are women who are influential and who boys can look up to as well,” People reports.","남아공 출신 공학자이자 기업가이며 위민 인 엔지니어링의 공동설립자인 두 아이의 엄마 나디야 무사지는 다음과 같이 말했다. “우리는 소녀들을 교육시키는 것뿐만 아니라 소년들의 롤모델이 되는 것도 중요하다는 걸 안다. 왜냐하면 우리는 영향력 있는 여성들이고, 소년들이 존경할 수 있는 사람들이기 때문이다.”라고 피플지(People)는 보도했다.","남아프리카의 엔지니어이자 사회적 기업가이자 여성 공학 공동 설립자인 Naadiya Mosajee는 ""우리는 여자아이들을 위한 교육뿐만 아니라 남자아이들을 위한 역할 모델이 되는 것의 중요성을 인정했습니다 – 우리는 영향력 있고 남자아이들도 존경할 수 있는 여자입니다,""라고 사람들은 보도했다.","남아프리카의 엔지니어이자 사회 사업가이자 공학 분야의 여성 공동 설립자인 Naadiya Moosajee는 다음과 같이 밝혔다. ""우리는 소녀를 위한 교육뿐만 아니라 소년을 위한 역할 모델의 중요성을 인정했습니다. 우리는 영향력 있고 소년이 존경할 수 있는 여성입니다.""라고 People은보고 합니다.",1,2,3,T,115.676923,120.225564


In [197]:
before[before["Segment ID"] == 280]

,Segment ID,Source Segment,Ref,MT_Y,MT_Z,Ref_rating,MT_Y_rating,MT_Z_rating,Error,MT_Y_TER,MT_Z_TER
279,280,"Citing a U.N.-backed reportreleased last year, demonstrators are demanding governments implement measures to prevent global temperatures from rising more than 2.7 degrees, which scientists warn result in dire consequences for human life and the planet.","지난해 공개된 UN 후원 보고서에 따르면, 시위자들은 지구 온도가 2.7도 이상 상승하지 않도록 정부가 적절한 조치를 취해야 한다고 요구하고 있다. 그렇지 않으면 인간의 삶과 지구 전체에 재앙을 초래할 결과를 초래한다고 과학자들은 말한다.","시위대는 지난해 유엔이 지지한 보고서를 인용, 지구온도가 2.7도 이상 상승하는 것을 막기 위한 조치들을 정부에 요구하고 있으며, 이는 인류와 지구에 심각한 결과를 초래할 것이라고 과학자들은 경고하고 있다.","시위대는 지난해 발표된 유엔의 지원을 받는 보고서를 인용, 지구 기온이 2.7도 이상 오르는 것을 막기 위한 조치를 시행할 것을 정부에 요구하고 있는데, 과학자들은 이 조치가 인간 생명과 지구에 심각한 결과를 초래한다고 경고하고 있다.",1,1,2,T,102.808989,112.0


In [187]:
after[after["Segment ID"] == 280]

,Segment ID,Source Segment,Ref,MT_Y,MT_Z,Ref_rating,MT_Y_rating,MT_Z_rating,Error,MT_Y_TER,MT_Z_TER
279,280,"Citing a U.N.-backed report released last year, demonstrators are demanding governments implement measures to prevent global temperatures from rising more than 2.7 degrees, which scientists warn result in dire consequences for human life and the planet.","지난해 공개된 UN 후원 보고서에 따르면, 시위자들은 지구 온도가 2.7도 이상 상승하지 않도록 정부가 적절한 조치를 취해야 한다고 요구하고 있다. 그렇지 않으면 인간의 삶과 지구 전체에 재앙을 초래할 결과를 초래한다고 과학자들은 말한다.","시위대는 지난해 유엔이 지지한 보고서를 인용, 지구온도가 2.7도 이상 상승하는 것을 막기 위한 조치들을 정부에 요구하고 있으며, 이는 인류와 지구에 심각한 결과를 초래할 것이라고 과학자들은 경고하고 있다.","시위대는 지난해 발표된 유엔의 지원을 받는 보고서를 인용해 지구 기온이 2.7도 이상 오르는 것을 막기 위한 조치를 시행할 것을 정부에 요구하고 있는데, 과학자들은 이를 통해 인간과 지구에 심각한 결과를 초래할 것이라고 경고하고 있다.",2,1,2,T,102.808989,112.0


In [194]:
a = before[before["Segment ID"] == 280].MT_Y
b = after[after["Segment ID"] == 280].MT_Y

a = [' '.join(mecab(s)).replace("▃", "").replace("  ", " ") for s in a]
b = [' '.join(mecab(s)).replace("▃", "").replace("  ", " ") for s in b]

In [195]:
sacrebleu.sentence_ter(a, b).score

0.0